# Test de l'API  YELP
## Import des revues de 200 business et sauvegarde dans un csv

In [1]:
# Pandas
import pandas as pd

# API
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from config import API_KEY_YELP

**La première requête retourne:**  
- 200 restaurants: le champ 'category' de l'objet 'business' doit contenir restaurant  
- d'une ville spécifiée dans la variable 'location'; ici san francisco  

**La deuxième requête retourne:** 
- les revues (3 max fournies par l'API) et leur rating pour chacun des 'business' sélectionné ci-dessus

**Les résultats sont sauvegardés dans un fichier csv dont les colonnes sont id (du business), revue, rating**

In [2]:
# Define the header
header = {'Authorization':f"bearer {API_KEY_YELP}",
         'Content-Type': 'application/json'}

# Build the request framework
transport = RequestsHTTPTransport(url = 'https://api.yelp.com/v3/graphql', headers = header, use_json = True)

# Create the client
client = Client(transport = transport, fetch_schema_from_transport = True)

######################################################
# Query: search the id of 200 restaurants in location
######################################################

# Init
biz = []
location = '"san francisco"'

for i in range(4):
    
    offset = 50 * i
    offset_text = "{offset: .0f}".format(offset=offset)
    query_text = '{search(categories: "restaurant",\
            location: ' + location + ',\
            offset: ' + offset_text + ',\
            limit: 50)\
            { business\
                 {  id\
                }\
            }\
        }'
    query = gql(query_text)
    resp = client.execute(query)
    biz.extend(resp['search']['business'])


######################################################
# Query: get the reviews for the passed business ids
######################################################

# Init
df_reviews = pd.DataFrame(columns=['id', 'review', 'rating'])
for b in biz:

    query_text = '{reviews(business: "' + b['id'] + '")\
                      {total\
                       review\
                          {text\
                           rating\
                          }\
                      }\
                  }'
    query = gql(query_text)
    resp = client.execute(query)

    # List of reviews for that business
    revs = [resp['reviews']['review'][i]['text'] for i in range(len(resp['reviews']['review']))]
    ratings = [resp['reviews']['review'][i]['rating'] for i in range(len(resp['reviews']['review']))]

    # DataFrame of reviews for that business

    revs_ = [[b['id'], revs[i], ratings[i]] for i in range(len(revs))]
    revs_ = pd.DataFrame(revs_, columns = ['id', 'review', 'rating'])
    df_reviews = pd.concat([df_reviews, revs_], axis=0)
    
# Save df_reviews (business ids and reviews)
df_reviews.to_csv('P06_4_API.csv')

In [3]:
df_reviews

,id,review,rating
0,wGl_DyNxSv8KUtYgiuLhmA,Who doesn't enjoy ice cream in the winter? Oka...,5
1,wGl_DyNxSv8KUtYgiuLhmA,"Believe the hype, Bi-Rite Creamery is worth th...",5
2,wGl_DyNxSv8KUtYgiuLhmA,This creamery is worth the hype! We've visited...,5
0,lJAGnYzku5zSaLnQ_T6_GQ,Um. You can turn any grits into cheddar/cheesy...,5
1,lJAGnYzku5zSaLnQ_T6_GQ,This was my favorite meal when I stayed in San...,5
...,...,...,...
1,OH3GLEJvGhbER4d3xMuPIA,Love it here. Adorable place. Such fine dining...,5
2,OH3GLEJvGhbER4d3xMuPIA,Covid review august 2020 \n\nLoved it here! \n...,5
0,Fe-YjAdSRLqDC2H_pqPcSw,"A+ Covid pick up order, ramen toppings separat...",5
1,Fe-YjAdSRLqDC2H_pqPcSw,5/5 - I love Hinodeya! I first heard of this p...,5
